Calculate %Jobs Accessible by Transit
PC

In [37]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
import numpy as np
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")   
NAMESPACE = "inro.emme.data.matrix.export_matrix_to_csv"
export_matrices = m.Modeller().tool(NAMESPACE)

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    
#obtain the minimum transit travel time between all modes.
#uses outputs from HDR transit time toolbox
mcSpec = {
    "expression": None,
    "result": "",
    "constraint": {
        "by_value": None,
        "by_zone": None
    },
    "aggregation": {
        "origins": None,
        "destinations": None
    },
    "type": "MATRIX_CALCULATION"
}

def minimum_transit_time(tt_dir, min_transit_time,scenario,go_sub_mtx_nums):
    
    #import the .311 files containing GO Train and subway OD travel times
    for root, dirs, files in os.walk(tt_dir):
        for matrix_file in files:
            if matrix_file.endswith(".311"):
                matrix_transaction(os.path.join(root, matrix_file),scenario)
                
    create_matrix(matrix_id = min_transit_time,
        matrix_name= "min.transit.time",
        matrix_description = "minimum.transit.time")

    matrix_list ={ "express.bus.actual": "",
                   "rapid.bus.actual":"",
                   "local.bus.actual":""
                   }

    #Import matrices
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name,
                      matrix_description = matrix_name)

    import_matrices_from_directory(tt_dir, matrix_list, ".mdf", scenario)
    
    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  "(" + matrix_list["express.bus.actual"] + ".min."\
            + matrix_list["rapid.bus.actual"] + ".min."\
            + matrix_list["local.bus.actual"] + ")"
    }
    matrix_calculator(spec)

    for mtx_num in go_sub_mtx_nums:
        expression = min_transit_time + ".min.mf" + mtx_num + "*(mf" + mtx_num + "!=0)+" + min_transit_time + "*(mf" + mtx_num + "==0)"
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  expression
        }
        matrix_calculator(spec)

    print "Finished Calculating OD Transit Travel Times"
    
def minimum_transit_time_mto(tt_dir,mto_skims, min_transit_time, scenario):            
    create_matrix(matrix_id = min_transit_time,
        matrix_name= "min.transit.time",
        matrix_description = "minimum.transit.time")

    matrix_list ={"express.bus.actual": "",
                   "rapid.bus.actual":"",
                   "local.bus.actual":""
                   }
    
    mat_list_mto = {"Express Bus": "",
                   "Local Bus": "",
                   "Rapid Bus": "",
                   "Subway": "",
                   "Train": ""}

    matching = {"Express Bus": "express.bus.actual",
               "Local Bus":"local.bus.actual",
               "Rapid Bus":"rapid.bus.actual"}
    #Import matrices
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name,
                      matrix_description = matrix_name)

    import_matrices_from_directory(tt_dir, matrix_list, ".mdf", scenario)
    
    for matrix_name in mat_list_mto.keys():
        mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = mat_list_mto[matrix_name],
                  matrix_name= matrix_name.replace(" ","_"),
                  matrix_description = matrix_name)

    import_matrices_from_directory(mto_skims, mat_list_mto, ".mdf", scenario)
    
    for match in matching.keys():
        mto = mat_list_mto[match]
        hdr = matrix_list[matching[match]]
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": mto,
           "expression":  hdr,
             "constraint": {
        "by_value": {
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE",
            "od_values": mto
        },
        "by_zone": None}
            }
        matrix_calculator(spec)
    min_exp = "("
    for matrix_name in mat_list_mto.keys():
        mat_id = mat_list_mto[matrix_name]
        min_exp = min_exp + mat_id + ".min."
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": mat_id,
           "expression":  "9999",
             "constraint": {
        "by_value": {
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE",
            "od_values": mat_id
        },
        "by_zone": None}
            }
        matrix_calculator(spec)
    min_exp = min_exp[0:-5] + ")"

    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":min_exp}
    matrix_calculator(spec)        
    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  "0",
        "constraint": {
        "by_value": {
            "interval_min": 9999,
            "interval_max": 9999,
            "condition": "INCLUDE",
            "od_values": min_transit_time
        },
        "by_zone": None}
            }
    matrix_calculator(spec)
    print "Finished Calculating OD Transit Travel Times"

def get_mc_spec(result, expression, agg_o, agg_d, con_o, con_d, con_val):
    specification = {
        "type": "MATRIX_CALCULATION",
        "result": result,
        "expression":  expression,
        "aggregation": {
            "origins": agg_o,
            "destinations": agg_d
        },
        "constraint": {
            "by_value": con_val,
            "by_zone": {
                "origins": con_o,
                "destinations": con_d
            }
        }
    }
    return specification

def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)

In [38]:
pop_o_matrix_ogs = "mo19"
emp_o_matrix_ogs = "mo20"
senior_o_matrix_ogs = "mo21"
low_inc_o_matrix_ogs = "mo22"# number of low inc households
youth_o_matrix_ogs = "mo23"
pop_d_matrix_ogs = "md92"
emp_d_matrix_ogs = "md93"
low_income_zones_ogs = "mo24"

tt_dir = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Peak"
tt_dir_op = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Off-Peak"
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"
local_fare_mat = "mf180"

In [39]:
sc_2051_BAU_ogs = "2051\\Short List\\BAU\\BAU_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201127"
sc_2051_prefnet_ogs = "2051\\Preferred Network\\Preferred_Network_SList_APTG_2051_GGH_MTP_V106A3_CS222_20210305"

scens = {
        "2051_prefnet_OGS":[sc_2051_prefnet_ogs,[701,702,703],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                 emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_BAU_OGS":[sc_2051_BAU_ogs,[404,405,406],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]]
        } #AM,MD,PM

In [41]:
for scen in scens.keys():
    print("Now processing {}".format(scen))
    scenario_folder = scens[scen][0]
    matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs" + "\\" + scenario_folder
    mto_tt_skims = matrix_folder + "\\Peak minimum skims LVL3_PATH\\time_all"
    mto_tt_skims_op = matrix_folder + "\\Offpeak minimum skims LVL3_PATH\\time_all"
    pop_o_matrix = scens[scen][2][0]
    emp_o_matrix = scens[scen][2][1]
    senior_o_matrix = scens[scen][2][2]
    low_inc_o_matrix = scens[scen][2][3]
    youth_o_matrix = scens[scen][2][4]
    pop_d_matrix = scens[scen][2][5]
    emp_d_matrix = scens[scen][2][6]
    low_income_zones = scens[scen][2][7]
    
    sc_num_am = scens[scen][1][0]
    sc_num_md = scens[scen][1][1]
    sc_num_pm = scens[scen][1][2]
    #scenarios
    sc_am = eb.scenario(sc_num_am)
    sc_md = eb.scenario(sc_num_md)
    sc_pm = eb.scenario(sc_num_pm)
    #networks
    ntwk_am = sc_am.get_network()
    ntwk_md = sc_md.get_network()
    ntwk_pm = sc_pm.get_network()
    
    clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")
    
    min_transit_time_peak = "mf140"
    
    #minimum transit time stuff
    create_matrix(matrix_id = min_transit_time_peak,
        matrix_name= "min.transit.time",
        matrix_description = "minimum.transit.time")

    mat_list_mto = {"All Transit": ""}

    for matrix_name in mat_list_mto.keys():
        mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = mat_list_mto[matrix_name],
                  matrix_name= matrix_name.replace(" ","_"),
                  matrix_description = matrix_name)

    import_matrices_from_directory(mto_tt_skims, mat_list_mto, ".mdf", sc_am)

    spec = {
       "type": "MATRIX_CALCULATION",
       "result": min_transit_time_peak,
       "expression":  mat_list_mto["All Transit"],
         "constraint": {
    "by_value": None,
    "by_zone": None}
        }
    matrix_calculator(spec)

    spec2 = {
       "type": "MATRIX_CALCULATION",
       "result": min_transit_time_peak,
       "expression":  hdr_bau_peak,
         "constraint": {
    "by_value": {
        "interval_min": 0,
        "interval_max": 0,
        "condition": "INCLUDE",
        "od_values":  min_transit_time_peak
    },
    "by_zone": None}
        }
    matrix_calculator(spec2)
    
    spec3 = {
       "type": "MATRIX_CALCULATION",
       "result": min_transit_time_peak,
       "expression":  hdr_bau_peak,
         "constraint": {
    "by_value": {
        "interval_min": 0,
        "interval_max": 800,
        "condition": "EXCLUDE",
        "od_values":  min_transit_time_peak
    },
    "by_zone": None}
        }
    matrix_calculator(spec3)

    clear_matrices(eb,[min_transit_time_peak,hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")

    min_transit_time_offpeak = "mf150"

    create_matrix(matrix_id = min_transit_time_offpeak,
        matrix_name= "min.transit.time.op",
        matrix_description = "minimum.transit.time")

    mat_list_mto = {"All Transit": ""}

    for matrix_name in mat_list_mto.keys():
        mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = mat_list_mto[matrix_name],
                  matrix_name= matrix_name.replace(" ","_"),
                  matrix_description = matrix_name)

    import_matrices_from_directory(mto_tt_skims_op, mat_list_mto, ".mdf", sc_md)

    spec = {
       "type": "MATRIX_CALCULATION",
       "result": min_transit_time_offpeak,
       "expression":  mat_list_mto["All Transit"],
         "constraint": {
    "by_value": None,
    "by_zone": None}
        }
    matrix_calculator(spec)

    spec2 = {
       "type": "MATRIX_CALCULATION",
       "result": min_transit_time_offpeak,
       "expression":  hdr_bau_offpeak,
         "constraint": {
    "by_value": {
        "interval_min": 0,
        "interval_max": 0,
        "condition": "INCLUDE",
        "od_values":  min_transit_time_offpeak
    },
    "by_zone": None}
        }
    matrix_calculator(spec2)
    
    spec3 = {
       "type": "MATRIX_CALCULATION",
       "result": min_transit_time_offpeak,
       "expression":  hdr_bau_offpeak,
         "constraint": {
    "by_value": {
        "interval_min": 0,
        "interval_max": 800,
        "condition": "EXCLUDE",
        "od_values":  min_transit_time_offpeak
    },
    "by_zone": None}
        }
    matrix_calculator(spec3)

    spec = get_mc_spec(None,emp_o_matrix,"+",None,None,None,None)
    total_emp = matrix_calculator(spec)['result']
    
    ### METHOD 1
    clear_matrices(eb,[min_transit_time_peak,min_transit_time_offpeak, hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")
    expression_access = "("+emp_d_matrix + ")*(" + min_transit_time_peak  + " <45)/" + str(total_emp) 
    spec = get_mc_spec("mo99", expression_access, None, "+", None, None, None)
    matrix_calculator(spec)
    
#     ### METHOD 2
#     expression_access = pop_o_matrix + "("+emp_d_matrix + ")*(" + min_transit_time_peak  + " <45)"
#     spec = get_mc_spec("mo99", expression_access, None, "+", None, None, None)
#     matrix_calculator(spec)
#     expression_total = pop_o_matrix + "*("+ emp_d_matrix + ")"
#     spec = get_mc_spec("mo98", expression_total, None, "+", None, None, None)
#     matrix_calculator(spec)
     
    export_matrices(matrices = ["mo99"], export_path = r"C:\Users\pechen\Desktop\Transit Accessibility\test\{}".format(scen))

Now processing 2051_BAU_OGS
Imported matrix mf1
Finished Importing Matrices
Imported matrix mf1
Finished Importing Matrices
Now processing 2051_prefnet_OGS
Imported matrix mf1
Finished Importing Matrices
Imported matrix mf1
Finished Importing Matrices
